<a href="https://colab.research.google.com/github/Daniil71/analysis_researches/blob/main/Numpy_scrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
pd.set_option('max_colwidth', 300)

Задание 1

In [ ]:

def posts_from_habr_page(url):
    """
        возвращает df c новостными постами с url-страницы хабра(параметр),
        в заголовках которых есть слова, переданные в keywords
    """
    my_keywords = [ 'microsoft', 'linux', 'ит', 'github', 'python', 'android']

    print('Введите 1, чтобы искать статьи по готовому списку, или 2, чтобы создать свой список.')
    print('Список слов для поиска по умолчанию: ', my_keywords)
    if int(input()) == 2:
       my_keywords = [ x.lower() for x in input('Введите слова для поиска: ').split()]    
    else:       
       pass

    req = requests.get(url).text
    habr_soup = BeautifulSoup(req)

    news = habr_soup.find_all('article', class_ = 'tm-articles-list__item')

    habr_df = pd.DataFrame()
    for article in news:
        title = article.find('a', 'tm-article-snippet__title-link').text
        for word in my_keywords:
            if word in title.lower().split():
                title_needed = title
                link ='http://habr.com' + article.find('a', 'tm-article-snippet__title-link').get('href')
                date = article.find('time').get('title')
                row = { 'date': date, 'title': title_needed, 'link': link }
                habr_df = pd.concat( [ habr_df, pd.DataFrame([row]) ] ).reset_index(drop = True)
    return habr_df, my_keywords

In [ ]:
habr_df, new_keywords = posts_from_habr_page('https://habr.com/ru/all/')
print('\nСписок слов: ', new_keywords)
habr_df

Введите 1, чтобы искать статьи по готовому списку, или 2, чтобы создать свой список.
Список слов для поиска по умолчанию:  ['microsoft', 'linux', 'ит', 'github', 'python', 'android']
2
Введите слова для поиска: api

Список слов:  ['api']


,date,title,link
0,"2022-02-04, 23:19",Версионирование API в ASP.Net Core,http://habr.com/ru/post/649827/


Задание 1.2

In [ ]:
def text_finder(row, my_keywords):
        """
            возвращает текст статьи из habr_df в столбец text, если в заголовке И самой статье есть слово из keywords
        """

        req = requests.get(row['link']).text
        soup = BeautifulSoup(req)
        full_text = soup.find('div', class_ = 'tm-article-body').text.strip()
        for word in my_keywords:
            # если не сделать проверку на текст И заголовок, то могут возникать забавные ситуации:
            # слово есть в заголовке, в статье его при этом нет, но в ней же есть другое слово из keywords
            if word in full_text.lower() and word in row['title'].lower():
                return full_text
        else:
            return 'strange text with no words from header'

In [ ]:
habr_df['text'] = habr_df.apply(text_finder, my_keywords = new_keywords, axis = 1)
habr_df

,date,title,link,text
0,"2022-02-04, 23:19",Версионирование API в ASP.Net Core,http://habr.com/ru/post/649827/,"Поддерживая существующие уже какое-то время Web API проекты, мы нередко сталкиваемся с проблемой устаревания логики методов контроллеров и необходимостью ее изменения в соответствии с новыми требованиями. Но, как правило, на момент возникновения такой необходимости, уже существует определенное ч..."


Задание 2

In [ ]:

def leaked_passwords_check(*emails):
    """
    Возвращает df с информацией о дате, источнике, описании взлома и email'e с api сайта avast.com/hackcheck.
    Email'ы передаются в качестве параметров через запятую в кавычках.
    """
    breaches_df = pd.DataFrame()
    for email in emails:
        url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
        my_json = {'emailAddresses': [email]}
        my_headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36',
            'Vaar-Header-App-Build-Version': '1.0.0',
            'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
            'Vaar-Version': '0'
        }
        res = requests.post(url, headers = my_headers, json = my_json)
        res.status_code
        my_res = res.json()
        

        for value in my_res['breaches'].values():
            source = value['site']
            date = value['publishDate']
            description = value['description']
            email_checked = " ".join(my_res['summary'])
            row = {'date': date, 'source': source, 'description': description, 'email_checked': email_checked }
            breaches_df = pd.concat( [ breaches_df, pd.DataFrame([row]) ]).reset_index( drop = True)
             
    return breaches_df

In [ ]:
leaked_passwords_check( 'xxx@x.ru','yyy@y.com')

,date,source,description,email_checked
0,2017-02-14T00:00:00Z,parapa.mail.ru,"In July and August 2016, two criminals executed attacks against three separate forums hosted by Mail.ru including the Russian forum Parapa. Shortly after the breach occurred, the contents of Parapa's database were leaked publicly. The database contains usernames, email addresses, and hashed pass...",xxx@x.ru
1,2016-10-29T00:00:00Z,vk.com,"Popular Russian social networking platform VKontakte was breached in late 2012. Over 100 million clear-text passwords were compromised in the breach. Breached credential sets included victims' e-mail addresses, passwords, dates of birth, phone numbers and location details. The credential set was...",xxx@x.ru
2,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe's corporate network and the stole source code for several of its software products. The affected products included Adobe's ColdFusion web application platform as well as the Acrobat suite of products. Adobe asserts that criminals also accessed nearl...",xxx@x.ru
3,2017-02-14T00:00:00Z,cfire.mail.ru,"In July and August of 2016, two criminals carried out attacks on three separate forums hosted by Mail.ru, including CFire. The hackers used known SQL injection vulnerabilities found in older vBulletin forum software to obtain access to the databases. Shortly after the breach itself, the contents...",xxx@x.ru
4,2017-01-31T00:00:00Z,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum database was breached. The attacker may have exploited a vulnerability in vBulletin. The stolen data contains over 1,870,000 user records including associated usernames, email addresses, hashed passwords, and salts. The leaked credentials are being sha...",xxx@x.ru
5,2016-10-23T00:00:00Z,imesh.com,"In June 2016, a cache of over 51 million user credentials from the online sharing service iMesh appeared for sale on a dark web marketplace. The database contained user email addresses, usernames, passwords, IP addresses and location.",xxx@x.ru
6,2020-01-03T00:00:00Z,azcentral.com,"At an unconfirmed date, online Arizona newspaper ""AZ Central"" was allegedly breached. The stolen data contains passwords and email addresses. This breach is being publicly shared on the internet.",yyy@y.com
7,2021-02-11T00:00:00Z,forums.vkmonline.com,"At an unconfirmed date, the Russian-language music forum VKM was allegedly breached. The stolen data contains passwords, IPs, email addresses, usernames and additional personal information. This breach is being privately shared on the internet.",yyy@y.com
8,2019-06-13T00:00:00Z,canva.com,"In May 2019, graphic-design site Canva's database was allegedly breached. The stolen data contains usernames, passwords, email addresses and additional personal information. This breach is being privately shared on the internet.",yyy@y.com
9,2016-10-24T00:00:00Z,dropbox.com,"Cloud storage company Dropbox suffered a major data breach in 2012. Criminals leveraged stolen employee login credentials to access a project document containing user email addresses and hashed passwords. Thanks to employees' passwords reuse between work and personal accounts, several matches we...",yyy@y.com


Задание 2.1

In [ ]:
from datetime import datetime

api_method = 'https://api.vk.com/method/wall.get'
TOKEN = '958eb5d439726565e9333aa30e50e0f937ee432e927f0dbd541c541887d919a7c56f95c04217915c32008'
ID = '-30159897'
VERSION = '5.131'
DOMAIN = 'https://vk.com/netology'
SLEEP = 0.33
params = {
    'owner_id': ID,
    'domain': DOMAIN,
    'access_token': TOKEN,
    'v': VERSION,
    'count': 50
}

In [ ]:
new_res = requests.get(api_method, params)
new_res = new_res.json()
vk_netology_posts = pd.DataFrame()

In [ ]:
for item in new_res['response']['items']:
    date = datetime.utcfromtimestamp( int(item['date']) ).strftime('%Y-%m-%d %H:%M:%S')
    post_text = item['text']
    row = {'date': date, 'post_text': post_text }
    vk_netology_posts = pd.concat( [vk_netology_posts, pd.DataFrame([row])] ).reset_index( drop = True)

In [ ]:
vk_netology_posts

,date,post_text
0,2022-02-04 14:00:00,"Сервис по поиску вакансий Работа.ру выяснил, что больше половины россиян хотят работать удалённо, чтобы самим планировать день 😌\n\nА как относитесь к удалёнке вы?"
1,2022-02-06 08:41:00,"«Чувствуйте себя как дома» — так говорят, когда хотят, чтобы человеку было максимально комфортно и уютно. Мы задумались: а всегда ли наш дом — именно такое место? Как сделать его идеальным, со своим характером и душой? \n\nСобрали подборку книг, которые помогут создать интерьер своей мечты и пре..."
2,2022-02-05 09:01:00,"Прошерстили интернет и собрали свежие исследования:\n\n🔹 Усердие или опыт: россияне ответили, что важнее для продвижения по карьерной лестнице → https://netolo.gy/hBP \n\n🔹 Больше половины россиян хотят работать удалённо, чтобы самим планировать день → https://netolo.gy/hA8 \n\n🔹 SuperJob: домаш..."
3,2022-02-04 08:02:00,"Окееей, летс го:\n\n▪ 9 февраля, 17:00 — Таргетолог или SMM-менеджер: что выбрать и как начать карьеру\nОбъясним, чем отличаются специалисты, какие задачи предстоит решать и какие перспективы ждут на рынке → https://netolo.gy/hA3 \n\n▪ 10 февраля, 19:00 — Как придумать хороший заголовок: 5 приём..."
4,2022-02-03 15:21:00,"Легко ли начинать новую карьеру? С одной стороны, сейчас есть возможности изучать любое направление из любой точки мира. С другой — потом предстоит найти работу без опыта в новой сфере. \n\nВыпускница Нетологии Елена Суслова прошла этот путь: в разгар пандемии, находясь в другой стране. Рассказы..."
5,2022-02-03 08:01:00,"В видео рассказываем, как избежать дедлайнов в лайфлайне → https://netolo.gy/hB1"
6,2022-02-02 13:30:00,"Стоп, перестаньте листать ленту и попробуйте решить нашу очередную математическую задачу 👇"
7,2022-02-02 07:35:00,"1 марта стартует бесплатный курс «Первые шаги в JavaScript: создаём сайт и приложение» 🤳\n\nКаждый сайт, каждое веб-приложение, которым вы когда-либо пользовались, с большой вероятностью содержит JS-код. Этот язык входит в десятку популярнейших языков в рейтинге Tiobe, имеет простой синтаксис и ..."
8,2022-02-01 15:07:00,"Хорошо, когда работа приносит только удовольствие: начальник хвалит, а коллеги всегда поддерживают и помогают. Но статистика говорит о том, что больше половины людей сталкиваются с конфликтами на работе минимум раз в месяц. \n\nРазбираемся, почему возникают рабочие конфликты, как вести себя в ни..."
9,2022-02-01 08:19:00,Не халтурит ли PR? Работает ли он? Какой у него график? \n\nВсё рассказываем здесь → https://netolo.gy/hA0
